# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [188]:
# import required libraries
import numpy as np
import pandas as pd

Next, import `Orders` from Ironhack's database into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [189]:
orders = pd.read_csv('../data/orders_sample.csv')
orders.head()

,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,546084,22741,2011,3,3,11,funky diva pen,48,2011-03-09 11:28:00,0.85,14112,United Kingdom,40.80
1,545906,22557,2011,3,2,9,plasters in tin vintage paisley,12,2011-03-08 09:23:00,1.65,15764,United Kingdom,19.80
2,539475,22176,2010,12,7,14,blue owl soft toy,1,2010-12-19 14:41:00,2.95,16686,United Kingdom,2.95
3,572562,21889,2011,10,2,9,wooden box of dominoes,12,2011-10-25 09:07:00,1.25,13481,United Kingdom,15.00
4,549372,72741,2011,4,5,11,grand chocolatecandle,9,2011-04-08 11:28:00,1.45,14958,United Kingdom,13.05


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [221]:
orders_id = orders.groupby(by='CustomerID').sum()
orders_id.head()

,InvoiceNo,year,month,day,hour,Quantity,UnitPrice,amount_spent
CustomerID,,,,,,,,
12347,3393172,12066,47,13,75,66,18.70,149.90
12348,1620634,6031,25,10,48,192,1.69,75.36
12349,2888045,10055,55,5,45,56,45.41,100.09
12350,543037,2011,2,3,16,12,0.85,10.20
12352,2284754,8044,40,13,56,14,54.83,126.48


In [222]:
quant_95 = orders_id['amount_spent'].quantile(q=0.95)
quant_75 = orders_id['amount_spent'].quantile(q=0.75)

is_preferred = (orders_id['amount_spent'] > quant_75) & (orders_id['amount_spent'] < quant_95)
is_vip = (orders_id['amount_spent'] > quant_95)
is_client = ~(is_vip | is_preferred)

orders_id = orders_id['amount_spent']

In [223]:
orders_id = orders_id.mask(is_vip, 'VIP').mask(is_preferred, 'Preferred').mask(is_client, 'Client')
orders_id = pd.DataFrame(orders_id)
orders_id.rename(columns={'amount_spent':'group'}, inplace=True)
group_clients = pd.merge(left=orders, right=orders_id, on='CustomerID')
group_clients

,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent,group
0,546084,22741,2011,3,3,11,funky diva pen,48,2011-03-09 11:28:00,0.85,14112,United Kingdom,40.80,Preferred
1,546084,21239,2011,3,3,11,pink polkadot cup,8,2011-03-09 11:28:00,0.85,14112,United Kingdom,6.80,Preferred
2,573406,23582,2011,10,7,15,vintage doily jumbo bag red,10,2011-10-30 15:57:00,2.08,14112,United Kingdom,20.80,Preferred
3,553940,23209,2011,5,5,10,lunch bag doiley pattern,10,2011-05-20 10:36:00,1.65,14112,United Kingdom,16.50,Preferred
4,573406,23344,2011,10,7,15,jumbo bag 50's christmas,10,2011-10-30 15:57:00,2.08,14112,United Kingdom,20.80,Preferred
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,550960,21249,2011,4,4,16,woodland height chart stickers,6,2011-04-21 16:39:00,2.95,12622,Germany,17.70,Client
19996,542470,22938,2011,1,5,11,cupcake lace paper set 6,12,2011-01-28 11:27:00,1.95,14133,United Kingdom,23.40,Client
19997,562570,23169,2011,8,7,12,classic glass cookie jar,1,2011-08-07 12:10:00,4.15,14079,United Kingdom,4.15,Client
19998,575949,23012,2011,11,7,11,glass apothecary bottle perfume,36,2011-11-13 11:51:00,3.45,16711,United Kingdom,124.20,Preferred


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [243]:
vips = group_clients.loc[group_clients['group'] == 'VIP', :]
vips.groupby(by='Country').count()['CustomerID'].sort_values(ascending=False).head(1)

Country
United Kingdom    3562
Name: CustomerID, dtype: int64

## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [249]:
vip_preferred = group_clients.loc[(group_clients['group'] == 'VIP') | (group_clients['group'] == 'Preferred'), :]
vip_preferred.groupby(by='Country').count()['CustomerID'].sort_values(ascending=False).head(1)

Country
United Kingdom    10094
Name: CustomerID, dtype: int64